# Referenced Mosaic

# Setup

## Imports

In [ ]:
import os

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [ ]:
from night_horizons import utils, preprocess, mosaic

## Settings

In [ ]:
settings = {
    'referenced_image_dir': '/Users/Shared/data/nitelite/images/220513-FH135/referenced',
    'mosaic_filepath': '/Users/Shared/data/nitelite/mosaics/220513-FH135/referenced2.tiff',
    'overwrite': True,
}

## Parse Settings

In [ ]:
referenced_fps = utils.discover_data(settings['referenced_image_dir'], ['tif', 'tiff'], pattern=r'Geo\s\d+_\d.tif')

# Extract/Transform/Load

In [ ]:
preprocessing = Pipeline([
    ('geotiff', preprocess.GeoTIFFPreprocesser()),
    ('geobounds', preprocess.GeoBoundsPreprocesser(passthrough=['filepath'])),
])

In [ ]:
X = preprocessing.fit_transform(
    referenced_fps,
)

# Build Mosaic

In [ ]:
if os.path.isfile(settings['mosaic_filepath']) and settings['overwrite']:
    os.remove(settings['mosaic_filepath'])

In [ ]:
model = mosaic.ReferencedMosaic(settings['mosaic_filepath'])

In [ ]:
model.fit_transform(X)

# Evaluate